# System


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 40.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
#clean_word
import re
from pythainlp.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
#Word_to_vector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
#model
from sklearn.linear_model import LogisticRegression
#split
from sklearn.model_selection import train_test_split
#Accuracy
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Data preprocessing

In [ ]:
#Load and preprocess the data
data = pd.read_csv('/content/drive/MyDrive/Project_IMDB/Colab train model/data/4_imdb_master.csv',encoding='ISO-8859-1')  # Replace 'shopping-comment.csv' with your dataset
data.shape
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  100000 non-null  int64 
 1   type        100000 non-null  object
 2   review      100000 non-null  object
 3   label       100000 non-null  object
 4   file        100000 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [ ]:
#Choose the colum and remove colum
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.drop(['file'], axis=1, inplace=True)
data.info()
data.head()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   type    100000 non-null  object
 1   review  100000 non-null  object
 2   label   100000 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


(100000, 3)

In [ ]:
# check dataset
print(data.columns)
print(data['label'].value_counts())
print(data[data['type'] == 'train'].value_counts())
data.isnull().values.any()

Index(['type', 'review', 'label'], dtype='object')
unsup    50000
neg      25000
pos      25000
Name: label, dtype: int64
type   review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

False

In [ ]:
# Cleaning the text
def cleaning(sen):
  sen = re.sub('[^A-Za-z]+', ' ', sen)
  sen = re.sub(r"\s+[a-zA-Z]\s+", ' ',sen)
  sen = re.sub(r'\s+',' ',sen)
  sen = re.sub(r'<[^>]+>', ' ',sen)
  return sen

data['review'] = data['review'].apply(cleaning)
data.head()

,type,review,label
0,test,Once again Mr Costner has dragged out movie fo...,neg
1,test,This is an example of why the majority of acti...,neg
2,test,First of all hate those moronic rappers who co...,neg
3,test,Not even the Beatles could write songs everyon...,neg
4,test,Brass pictures movies is not fitting word for ...,neg


In [ ]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

#set stopwords to english
stopword=set(stopwords.words('english'))
print(stopword)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

#Apply function on review column
data['review']=data['review'].apply(remove_stopwords)
data.head()

{"couldn't", 'our', 'themselves', 'do', 'any', 'shouldn', "doesn't", 'itself', 'we', 'ours', 'yourself', "hasn't", 'him', 'through', 'shan', 'y', 'you', 'those', 'herself', 'doing', 'doesn', "you've", "isn't", 'ma', 'did', 'in', "she's", 'all', "you'll", 'to', 'into', 'the', 'very', 'his', 'up', 'll', 'me', 'this', 'a', 'at', 'does', 'that', 'such', 'which', 'whom', 's', 've', 'off', 'too', 't', "it's", 'about', 'being', "aren't", 'over', 'aren', 'mightn', 'they', 'under', 'each', 'couldn', 'were', 'hers', 're', 'are', 'some', 'wasn', 'has', "shouldn't", 'here', 'them', 'not', 'ain', 'after', 'down', "won't", 'wouldn', "weren't", 'what', 'out', 'because', 'before', 'only', "haven't", 'won', 'of', 'no', "don't", 'where', 'there', 'had', 'then', 'so', 'below', 'again', "should've", 'from', 'my', 'more', 'haven', "needn't", 'he', 'and', 'if', "didn't", 'be', 'on', 'didn', "wouldn't", 'was', 'with', 'her', "that'll", 'for', 'its', 'is', 'why', 'these', 'same', 'can', 'when', 'who', 'hadn',

,type,review,label
0,test,Mr Costner dragged movie far longer necessary ...,neg
1,test,example majority action films Generic boring r...,neg
2,test,First hate moronic rappers could nt act gun pr...,neg
3,test,even Beatles could write songs everyone liked ...,neg
4,test,Brass pictures movies fitting word really some...,neg


In [ ]:
#variables
X = data['review']
y = data['label']

# Logistic Regression Model

In [ ]:
# Tokenization & Vectorization (TF-IDF)
tfidf = TfidfVectorizer(lowercase=True)

# Training set and Test set
X_train,X_test,Y_train,Y_test = train_test_split(X,y, test_size=0.20, random_state=225)

# Train Logistic Regression
logicRe =LogisticRegression(solver='lbfgs', max_iter=3000)
lr = Pipeline([('vectorizer',tfidf),('classifier',logicRe)])
lr.fit(X_train,Y_train)
lrpred = lr.predict(X_test)

# Accuracy
print("Accuracy of Logistic Regression => ",accuracy_score(lrpred,Y_test)*100)

Accuracy of Logistic Regression =>  62.975


# Prediction

In [ ]:
new_text = ["The comments already posted for this film do not do it justice. It is a very low budget sci-fi film made by lesser known people and should be judged that way. To say the film shouldn't be made is plain wrong. How do people learn without doing? Small pictures should be encouraged, not made fun of just because they are not of the highest quality."]
prediction = lr.predict(new_text)

# Displaying Results
for text, prediction_index in zip(new_text,prediction):
    if prediction_index == "pos":
        prediction_label = 1
    elif prediction_index == "unsup":
        prediction_label = 0
    else:
        prediction_label = -1

print("Predicted class:", prediction_label)

Predicted class: 0


In [ ]:
new_text = ["This is an enjoyable movie. Its very realistic to the wonderful world of music I've been there and done that. It shows a human element in each character and the realism that nobody is perfect. These amateur musicians weren't all that bad players. Cleavon Little's character, Marshall Tucker, was played very well. Marshall was no saint himself. Here he was getting paid to do a job and he's giving these guys a hard time about everything in the van on the way up there. You don't bite the hands that feed you. I do find it hard to believe that a player with the jazz experience he has, claims he does not know any of the dixieland tunes. He has a tremendous sense of predicting chord changes to tunes he does not know. Not common, but not unheard of either. He delivers a true and harsh message at the end of the movie when he tells the clarinet player, its not a religion, devotion is not enough. On that level, he is correct, although I think the clarinet player could have handled the job. He was practicing his butt off and vocal accompaniment music is not that hard to read. Very enjoyable movie."]
prediction = lr.predict(new_text)

# Displaying Results
for text, prediction_index in zip(new_text,prediction):
    if prediction_index == "pos":
        prediction_label = 1
    elif prediction_index == "unsup":
        prediction_label = 0
    else:
        prediction_label = -1

print("Predicted class:", prediction_label)

Predicted class: 1


# Pickle

In [ ]:
import pickle
with open('Logistic.pkl', 'wb') as file:
    pickle.dump(lr, file)